<a href="https://colab.research.google.com/github/LucasMirandaD/LucasMirandaD/blob/main/LM_Competencia_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ayuda -> Caso_Muerte_Coronaria_2023

In [1]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split

In [2]:
datos= pd.read_csv('https://raw.githubusercontent.com/profcarlosamartinez/mineria_datos/main/datos_m_coronaria_2023/datos_train_test_sh.csv',delimiter=',',decimal='.')

In [3]:
datos.head(2)

,id,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,clase
0,0,154,4.50,4.75,23.52,Present,43,25.76,0.0,53,1
1,1,124,1.04,2.84,16.42,Present,46,20.17,0.0,61,0


In [4]:
datos.shape

(346, 11)

In [5]:
datos.clase.value_counts()

0    226
1    120
Name: clase, dtype: int64

In [6]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         346 non-null    int64  
 1   sbp        346 non-null    int64  
 2   tobacco    346 non-null    float64
 3   ldl        346 non-null    float64
 4   adiposity  346 non-null    float64
 5   famhist    346 non-null    object 
 6   typea      346 non-null    int64  
 7   obesity    346 non-null    float64
 8   alcohol    346 non-null    float64
 9   age        346 non-null    int64  
 10  clase      346 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 29.9+ KB


- Aplicamos One-Hot Encoding cuando:

    La característica categórica no es ordinal
    El número de características categóricas es menor, por lo que puede aplicarse eficazmente la codificación de una sola vez. Se puede usar pandas con pd.get_dummies o from sklearn.preprocessing import OneHotEncoder.

In [7]:
# Get list of categorical variables
s = (datos.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['famhist']


In [8]:
# Usando OneHotEncoder de sklearn
# Para pasar variables categóricas nominales a números
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
oh_array = ohe.fit_transform(datos['famhist'].values.reshape(-1, 1)).toarray()
oh_df = pd.DataFrame(oh_array,columns=['famhist_Absent','famhist_Present'])
df=pd.concat([datos,oh_df],axis=1)
df.head(5)

#Otra forma de hacer OneHotEncoder
# dummies = pd.get_dummies(datos[['famhist']])
# datos.drop(['famhist'],axis=1)
# df = pd.concat([datos, dummies],axis=1)
# df.head(2)

,id,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,clase,famhist_Absent,famhist_Present
0,0,154,4.50,4.75,23.52,Present,43,25.76,0.00,53,1,0.0,1.0
1,1,124,1.04,2.84,16.42,Present,46,20.17,0.00,61,0,0.0,1.0
2,2,148,12.20,3.79,34.15,Absent,57,26.38,14.40,57,1,1.0,0.0
3,3,110,4.64,4.55,30.46,Absent,48,30.90,15.22,46,0,1.0,0.0
4,4,164,0.50,6.95,39.64,Present,47,41.76,3.81,46,1,0.0,1.0


In [9]:
datos = df.reindex(columns=['id','sbp','tobacco','ldl','adiposity','typea','famhist_Absent','famhist_Present','obesity','alcohol','age','clase'])
datos.head(2)

,id,sbp,tobacco,ldl,adiposity,typea,famhist_Absent,famhist_Present,obesity,alcohol,age,clase
0,0,154,4.50,4.75,23.52,43,0.0,1.0,25.76,0.0,53,1
1,1,124,1.04,2.84,16.42,46,0.0,1.0,20.17,0.0,61,0


In [10]:
X=datos.iloc[:,0:11] #antes --> X=datos.iloc[:,0:11] incluia el id
X.head(4)

,id,sbp,tobacco,ldl,adiposity,typea,famhist_Absent,famhist_Present,obesity,alcohol,age
0,0,154,4.50,4.75,23.52,43,0.0,1.0,25.76,0.00,53
1,1,124,1.04,2.84,16.42,46,0.0,1.0,20.17,0.00,61
2,2,148,12.20,3.79,34.15,57,1.0,0.0,26.38,14.40,57
3,3,110,4.64,4.55,30.46,48,1.0,0.0,30.90,15.22,46


In [11]:
y=datos.iloc[:,11:12]
y.head(4)

,clase
0,1
1,0
2,1
3,0


In [12]:
datos.info() #agregado para control

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               346 non-null    int64  
 1   sbp              346 non-null    int64  
 2   tobacco          346 non-null    float64
 3   ldl              346 non-null    float64
 4   adiposity        346 non-null    float64
 5   typea            346 non-null    int64  
 6   famhist_Absent   346 non-null    float64
 7   famhist_Present  346 non-null    float64
 8   obesity          346 non-null    float64
 9   alcohol          346 non-null    float64
 10  age              346 non-null    int64  
 11  clase            346 non-null    int64  
dtypes: float64(7), int64(5)
memory usage: 32.6 KB


**Dividir el conjunto de datos**

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=80)

In [14]:
type(X_train)

pandas.core.frame.DataFrame

In [15]:
type(y_train)

pandas.core.frame.DataFrame

**Subdivido el conjunto de entrenamiento**

In [16]:
X_train_v, X_test_v, y_train_v, y_test_v = train_test_split(X_train, y_train,test_size=0.25,random_state=80)

**Creo el modelo**

In [17]:
from sklearn.neighbors import KNeighborsClassifier
#  from sklearn import tree # Importado arriba

modelo = tree.DecisionTreeClassifier() # nuevo modelo de clasificacion

# modelo_knn = KNeighborsClassifier(n_neighbors=3)

**Entrenamiento del modelo**

In [18]:
# modelo_knn.fit(X,np.ravel(np.array(y)))
#  Entrenamiento del modelo 
modelo.fit(X_train_v,y_train_v)
# modelo.fit(X_train,y_train)
tree.DecisionTreeClassifier()

DecisionTreeClassifier()

Predicciones con el conjunto de validacion

In [19]:
# y_pred=modelo.predict(X_test_v)
# y_pred= pd.DataFrame(y_pred)
# y_pred.head(3)

**Calibrar el modelo**

In [20]:
from sklearn.model_selection import GridSearchCV

**Diccionario de hiperparametros**

In [21]:
tree_param = {
    'criterion': ['entropy','gini'], 
    'max_depth': range(1,10),
    'min_samples_leaf': range(1,10)
}

In [22]:
grid = GridSearchCV(
    modelo,
    param_grid= tree_param,
    cv=5,
    verbose=1
)
grid.fit(X_train_v,y_train_v)
# grid.fit(X_train,y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': range(1, 10),
                         'min_samples_leaf': range(1, 10)},
             verbose=1)

Encuentro los mejores parametros

In [23]:
grid.best_params_

{'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 8}

In [24]:
grid.best_score_

0.7232732732732734

In [25]:
grid.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=8)

Modelo final

In [26]:
modelo_f=grid.best_estimator_

In [27]:
modelo_f.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=8)

In [28]:
modelo_f

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=8)

**Leer nuevas instancias a predecir**

In [29]:
nuevas_instancias= pd.read_csv('https://raw.githubusercontent.com/profcarlosamartinez/mineria_datos/main/datos_m_coronaria_2023/nuevas_instancias_a_predecir.csv',delimiter=';',decimal='.')

In [30]:
ohe = OneHotEncoder()
oh_array = ohe.fit_transform(nuevas_instancias['famhist'].values.reshape(-1, 1)).toarray()
oh_df1 = pd.DataFrame(oh_array,columns=['famhist_Absent','famhist_Present'])
df1=pd.concat([nuevas_instancias,oh_df1],axis=1)
df1.head(2)

#Otra forma de hacer OneHotEncoder
#dummies = pd.get_dummies(nuevas_instancias[['famhist']])
#nuevas_instancias.drop(['famhist'],axis=1)
#df1 = pd.concat([nuevas_instancias, dummies],axis=1)

nuevas_instancias = df1.reindex(columns=['id','sbp','tobacco','ldl','adiposity','typea','famhist_Absent','famhist_Present','obesity','alcohol','age'])
nuevas_instancias.head(2)

,id,sbp,tobacco,ldl,adiposity,typea,famhist_Absent,famhist_Present,obesity,alcohol,age
0,1,132,0.0,4.17,36.57,57,1.0,0.0,30.61,18.00,49
1,2,124,1.8,3.74,16.64,42,0.0,1.0,22.26,10.49,20


In [31]:
prediccion_nuevos_individuos=modelo_f.predict(nuevas_instancias)

In [32]:
datos_clasificados=pd.DataFrame(prediccion_nuevos_individuos)

In [33]:
datos_clasificados.index = datos_clasificados.index + 1

In [34]:
datos_clasificados.columns = ['Predicted']
datos_clasificados.index.names = ['id']

In [35]:
datos_clasificados.columns

Index(['Predicted'], dtype='object')

In [36]:
datos_clasificados.head(10)

,Predicted
id,
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


**Validacion del modelo**

In [37]:
from sklearn.metrics import classification_report

y_pred = modelo_f.predict(X_test)
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.88      0.81        69
           1       0.64      0.40      0.49        35

    accuracy                           0.72       104
   macro avg       0.69      0.64      0.65       104
weighted avg       0.71      0.72      0.70       104



In [38]:
from google.colab import files

In [40]:
datos_clasificados.to_csv('predicted.csv',sep=',')
files.download('predicted.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>